# WHIN Supply Chain Web Scraper 

### Team Members 
1. Deepika Jindal 
2. Mihir Bhatia 

## Importing Packages 

In [ ]:
import urllib.request
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urltools
import time 
from pandas import ExcelWriter
from pandas import ExcelFile
import time

## Get base URL of the website 

This function lets you extract the base URL of the website. It will be used to check if a web page is from the original webiste that we intended to scrape. 

In [4]:
def get_base_url(url):
    from urllib.parse import urlsplit
    base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url))
    

    if base_url.startswith("https"):
        base_url = base_url.replace('https://','')
    elif base_url.startswith("http"):
        base_url = base_url.replace('http://','')
        
    if base_url.endswith('/'):
        base_url = base_url.replace('/','')
    return base_url


##  Does the page exist ? 
Our web scrapper will go the website and use the http protocol to check if the web page which we intend to scrape exists 

In [5]:
def does_page_exist(url):
    exists = 0 
    try:
        request = requests.get(url)
        if request.status_code == 200:
            exists = 1 
            
    except: 
        pass
    
    return exists



In [6]:
# Function to extract links from a URL------------------------------------------
def href_scrapper(url):

    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        a_tag = soup.find_all('a')
        img_tag = soup.find_all('img')
        link_tag = soup.find_all('link')
        link_list = list()

        for links in a_tag:
            link_list.append(links.get('href'))

        for links in link_tag:
            link_list.append(links.get('href'))

        for links in img_tag:
            link_list.append(links.get('src'))

        return link_list

    except:
        return 0

## What if the link extracted has no http in it ? 
There will be cases in which the link you extracted has no http in the beginning. This function adds an http to the beginning of the link to keep all links similar in structure. This will be useful during string manipulations and will standardize link structure 

In [7]:
# Function to create complete links---------------------------------------------
def link_former(url):
    x = url

    if x.startswith("http"):
        return x

    else:
        full_url = original_url + x
        return full_url



## Save files 
This function saves files with various formats. Our main web scrapper doesn't use all these extensions. But we provide this function incase somoene wants to scrape images, PDFs and webpages 

In [8]:
# Function to save links to directory of python file----------------------------
def file_save(url, i):
    save_error = 0
    full_url = url
    try:
        if str(full_url).endswith(".htm"):
            name = "WebPage" + str(i) + ".htm"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".pdf"):
            name = "Pdf" + str(i) + ".pdf"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".jpg"):
            name = "Image" + str(i) + ".jpg"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".jpeg"):
            name = "Image" + str(i) + ".jpeg"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".png"):
            name = "Image" + str(i) + ".png"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".css"):
            name = "CSS File" + str(i) + ".css"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        else:
            name = "WebPage" + str(i) + ".html"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        return m
    except:
        save_error = save_error + 1



## Now it's time to crawl
The crawl function takes a link, with a number ID and output file and outputs all the daughter links associated with that parent link. 

In [9]:
'''
crawl function = takes link as input and crawls relevant webpages from the parent 

Inputs 
----------

Outputs
----------
'''
def crawl(original_url, num_Id, output_file):
    
    # Accepting input of URL and depth----------------------------------------------
    tic = time.time()
    
    parent_list = [original_url]
    url_to_check = get_base_url(original_url)
    print(url_to_check)
    url_to_check = str(url_to_check)

    layer_stop = 1
    layer = 0


    #initializing all the requiered lists---------------------------------------------
    visited_all  = [original_url]
    visited_current_layer = []
    child_list =[]
    child_list_filtered = [original_url]
    #columns = ['Link','Parent Link', 'Layer']
    df = pd.DataFrame()

    Di={}

    # Main execution of scrapper----------------------------------------------------

    #looping through layers 
    while layer < layer_stop:
    
        #looping through URLs in parent-list
        for url in parent_list:
        
            #scraping the children from the parent url----------------------------
            if href_scrapper(url) != 0:
                child_list = href_scrapper(url)
    
            
            for child in child_list:
                if child != None:
                    ch=child
                    #if child link is of the form "index.php/blahblah" and parent ends with '/'
                    #---> "parentlink/index.php/blahblah"
                    if child.startswith('/'):
                        child= str(url) + str(child)
                    
                    if url.endswith('/') and url_to_check not in child:
                        child = str(url) + str(child)
                    
                #normalize the child links-------------------------------------
                    child=urltools.normalize(child)  
                    

                    #filtering out based on 1) External 2) Repeating 3) Invalid links---------------------------
                    if url_to_check in child and child not in visited_all and does_page_exist(child)==1 and ch not in Di:
                        child_list_filtered.append(child)
                        Di[ch]=1
                
                    #adding everthing to visited all--------------------
                    if child not in visited_all:
                        child_slash = child + '/'
                        visited_all.append(child)  
                        visited_all.append(child_slash)
                    
                    
            #adding  the visited and filtered children into the "current visited layer" ------ 
            for child_filtered in child_list_filtered:
                visited_current_layer.append(child_filtered)

            #creating a Pandas dataframe to store everything for download----------
            layer_number = [layer+1]*len(child_list_filtered)
            parent_of_child = [url]*len(child_list_filtered)

            df_child = pd.DataFrame(child_list_filtered)
            df_parent = pd.DataFrame(parent_of_child)
            df_layer = pd.DataFrame(layer_number)


            df_to_be_added = pd.concat([df_child,df_parent,df_layer], axis=1)
            df = pd.concat([df,df_to_be_added],ignore_index=True, axis = 0)
            #----------------------------------------------------------------------
        
            #emptying the child lists
            child_list = []
            child_list_filtered = []
        
        #condition to stop filtering-----------------------------------------------
        if not visited_current_layer :
            layer_stop = layer_stop 
        else:
            layer_stop += 1

    
        #child layer is now parent layer--------------------------------------------
        parent_list = []
    
        #we only dont add .png, .jpg , .pdf to the new parent layer 
        for visited_current in visited_current_layer: 
            print(visited_current)
            if(not visited_current.endswith('.png') and not visited_current.endswith('.jpg') and not  visited_current.endswith('.pdf')):
                parent_list.append(visited_current)
            
            
        #displaying the links in different layers----------------------------------
        #print("Links in LAYER:" + str(layer+1))
        print("No of links = " + str(len(visited_current_layer)))
        #print(visited_current_layer)
        print("\n")
        visited_current_layer = [] 
        #updating the layer number
        layer +=1
    df.to_csv(output_file + '/' + str(num_Id) + '_' + str(url_to_check) +  '.csv', sep=',', encoding='utf-8')
    return df, num_Id

## Functions that will be used to scrape the crawled webpages 

In [10]:
'''
Functions that will be used by the scraper function 
tag_visible = 
text_from_html = 
'''

import requests
page = requests.get("https://docs.python.org/3/howto/urllib2.html")
from bs4 import BeautifulSoup
from bs4.element import Comment
soup = BeautifulSoup(page.content, 'html.parser')


#information based on:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True
p_tag=soup.find_all('p')

#extract text from html: 
def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)
web_text = []

## Scrape the webpages 
This function allows us to scrape all the webpages inside the dataframe that contains the crawled links

In [11]:
import json
import urllib
import os 
extensions=('.png', '.jpg','.jpeg','.pdf', '.gif','.ico')
def scrape(df, scraped_website_path, website_index, website_name):
    TempD={}
    for index,item in df.iterrows():
        #print(item[1],item[0])
        try:      
            #for i in extensions:
            #print(item[1])
            if item[1].endswith(extensions):
                print(item[1],"skipped 1")
                
            elif ".gif&" in item[1]:
                print(item[1],"skipped 2")
                
                
            else:
                print(item[1],"printed")
                html = urllib.request.urlopen(item[1]).read()
                #web_text.append(text_from_html(html))
                print(text_from_html(html))
                time.sleep(3)
                #save_url(str(item[1]), index)
                TempD[item[1]]=text_from_html(html)
        #label: end
        except: 
            pass
    with open(scraped_website_path + '/' + 'json_' + str(website_index) + str(get_base_url(website_name)) +  '.json', 'w') as outfile:  
        json.dump(TempD, outfile)
        
    return 1

## Crawler - that works with an excel sheet containing all the relevant webpages 

In [10]:

# web_crawler_all() = Scraper for multiple websites# web_cr 
'''
Inputs
-----------
excel_path = Excel file path which contains list of websites to be crawled  
sheet = sheet name of this excel file 
web_col = the column no of in which the website hompages are located 
index_col = the column with indices of websites
parent_col = the column with the parent names
output_folder = path of the output folder where we wish to save the websites 
from_website = the index no. from website (typically starting from 1)
to_website = the index no of the website till we want to continue scraping (scraped websites will include the 
                websites with this index)

Output 
-----------
CSV files containing a list of child links for each website are saved into the output_folder path 

'''
def web_crawler_all(excel_path, sheet, web_col, index_col, parent_col,  output_folder, from_website, to_website):
    df = pd.read_excel(excel_path , sheet_name=sheet)
    df = df.iloc[from_website-1 : to_website, :]
    print(df)
    for index,row in df.iterrows():
        num_Id=int(row[index_col])
        original_url=row[web_col]

        
        try:
            websites= pd.read_csv(output_folder + str(int(num_Id)) + '_' + str(original_url) + '.csv')
            print("website is already crawled")
        except:
            websites, num_Id = crawl(original_url ,num_Id ,output_folder)
            
        
    print("All links from the websites indicated have been crawled")
    print("Output File :  " + str(output_folder))
    print("No. of websites scraped : " + str(to_website - from_website + 1))
    return 1

### Things to be added
- index to track what has been scrolled 
- time taken by the href function
- file structure for all webpages


## Scraper that works with a file containing all the scrapped webpages in an excel sheet

In [12]:
'''
Inputs 
----------
excel_path = Excel file path which contains list of websites to be crawled  
sheet = sheet name of this excel file 
crawled_website_path = path of file where all the crawled websites are stored  
scraped_website_path = path of file where all the scraped websites are stored
from_website - to_website = website index number from-to which we want to scrape and store in JSON 
web_col = column number of websites (0,1,2....)

Output
----------
JSON files for each parent website in scraped_website_path folder. The path of the scraped websites 
is the same as that crawled websites 
'''

def web_scraper_all(df, crawled_website_path, scraped_website_path, from_website, to_website,web_col):
    #get excel sheet containing websites 
  #  df = pd.read_excel(excel_path , sheet_name=sheet)
    df = df.iloc[from_website-1 : to_website, :]   
    
    #loop over all the websites and extract child links. Store them in JSON files 
    
    for index, rows in df.iterrows():
        #setting the website index and name 
        website_index = int(rows[0] )
        website_name = rows[web_col]
        
        #generating filename and getting the csv file of child links 
        filename = crawled_website_path + '/'+ str(website_index)+ '_' + str(get_base_url(website_name)) + ".csv"
        df_of_child_links = pd.read_csv(filename)
        print(filename,df_of_child_links )
        #executing the scrape function 
        flag = scrape(df_of_child_links , scraped_website_path, website_index, website_name) 
        
        #checkpoint to see if scraping is complete 
        if flag == 1 :
            print(website_name + ' has been scraped successfully')
            #print('scraping percentage complete = ' + str((i/to_website)*100) + '%')
            flag = 0

## Excection of the scrapper_all function 

In [15]:
#executing the scraper function 

excel_path = 'batch1.xlsx'
sheet = 'Sheet1'
df = pd.read_excel(excel_path , sheet_name=sheet)
crawled_website_path = 'Crawled'
scraped_website_path = 'Scraped'
from_website = 1
to_website = 1
web_col = 2 
df = pd.read_excel(excel_path , sheet_name=sheet)
#print(df)
web_scraper_all(df, crawled_website_path, scraped_website_path, from_website, to_website,web_col)

Crawled/6_www.oerlikon.com.csv      Unnamed: 0                                                  0  \
0             0  https://www.oerlikon.com/en/company/company-ov...   
1             1  https://www.oerlikon.com/en/company/company-ov...   
2             2  https://www.oerlikon.com/fairfield/en/about-oe...   
3             3  https://www.oerlikon.com/fairfield/en/media/pr...   
4             4  https://www.oerlikon.com/fairfield/en/contact/...   
5             5  https://www.oerlikon.com/fairfield/en/media/do...   
6             6  https://www.oerlikon.com/fairfield/en/contact/...   
7             7  https://www.oerlikon.com/fairfield/en/services...   
8             8             https://www.oerlikon.com/en/media/rss/   
9             9  https://www.oerlikon.com/ecomaXL/files/fairfie...   
10           10  https://www.oerlikon.com/en/company/company-ov...   
11           11                   https://www.oerlikon.com/balzers   
12           12                     https://www.oerlikon.co

                 \uf1ad         Company Company Profile Company overview Organization Facts & Figures Market Segments Global footprint Compliance Tax strategy Sponsoring History Supplier Portal Contact Innovation Our Competencies Our Markets Research and Development Customer Reference Quality and Certificates Did you know 24h Trip 24h Around the World Oerlikon Blog Media Press Releases Events and Agenda Media Gallery Newsroom Subscribe RSS-Newsfeed Contact Newsletter Investor Relations Factsheet Corporate Governance Reports and Publications Share Information Creditor Relations Financial Calendar General Meeting Contact and Services Career Vacancies Why to join us How to join us Your Career with us Find your fit Our employees Contact Contact Headquarter Request form Worldwide        Group    Corporate    Surface Solutions Segment   Balzers  Metco  Additive Manufacturing     Manmade Fibers Segment   Barmag  Neumag     Drive Systems Segment   Graziano  Fairfield        Region   China  Ind

https://www.oerlikon.com/en/company/company-overview/segments-drivesystems/ printed
                 \uf1ad         Company Company Profile Company overview Organization Facts & Figures Market Segments Global footprint Compliance Tax strategy Sponsoring History Supplier Portal Contact Innovation Our Competencies Our Markets Research and Development Customer Reference Quality and Certificates Did you know 24h Trip 24h Around the World Oerlikon Blog Media Press Releases Events and Agenda Media Gallery Newsroom Subscribe RSS-Newsfeed Contact Newsletter Investor Relations Factsheet Corporate Governance Reports and Publications Share Information Creditor Relations Financial Calendar General Meeting Contact and Services Career Vacancies Why to join us How to join us Your Career with us Find your fit Our employees Contact Contact Headquarter Request form Worldwide        Group    Corporate    Surface Solutions Segment   Balzers  Metco  Additive Manufacturing     Manmade Fibers Segment   Barma

https://www.oerlikon.com/fairfield/en/about-oerlikon-fairfield/company-overview/ printed
                 \uf1ad         About Oerlikon Fairfield Company Overview Our Competencies Applications Worldwide Locations Historical Milestones Quality & Certificates Employees & Career Employee Statements Products Custom Gears Custom Assemblies Torque Hub® Drives Torque Lock™ Brakes Ashwoods Electric Motors Services Service Policies & Manuals Engineering Services Media Press Releases Events Downloads Publications Contact Contact For Suppliers For Product & Application Support General Inquiry Request for Quote Locations Worldwide        Group    Corporate    Surface Solutions Segment   Balzers  Metco  Additive Manufacturing     Manmade Fibers Segment   Barmag  Neumag     Drive Systems Segment   Graziano  Fairfield         Search                You are here:   Home ›  About Oerlikon Fairfield › Company Overview     About Oerlikon Fairfield   Company Overview Our Competencies Applications Worldwide

https://www.oerlikon.com/fairfield/en/contact/request-form/ printed
                 \uf1ad         About Oerlikon Fairfield Company Overview Our Competencies Applications Worldwide Locations Historical Milestones Quality & Certificates Employees & Career Employee Statements Products Custom Gears Custom Assemblies Torque Hub® Drives Torque Lock™ Brakes Ashwoods Electric Motors Services Service Policies & Manuals Engineering Services Media Press Releases Events Downloads Publications Contact Contact For Suppliers For Product & Application Support General Inquiry Request for Quote Locations Worldwide        Group    Corporate    Surface Solutions Segment   Balzers  Metco  Additive Manufacturing     Manmade Fibers Segment   Barmag  Neumag     Drive Systems Segment   Graziano  Fairfield         Search                You are here:   Home ›  Contact › General Inquiry     Contact   For Suppliers For Product & Application Support General Inquiry Request for Quote Locations Worldwide      Gener

...

https://www.facebook.com/sharer/sharer.php?ars=facebook_login&locale2=en_US&lwv=100&refid=9&u=https://www.oerlikon.com/fairfield/en/services/service-policies//recover/initiate/?c=https://m.facebook.com/login.php?skip_api_login%3D1%26api_key%3D966242223397117%26signed_next%3D1%26next%3Dhttps%3A%2F%2Fm.facebook.com%2Fsharer%2Fsharer.php%3Fu%3Dhttps%253A%252F%252Fwww.oerlikon.com%252Ffairfield%252Fen%252Fservices%252Fservice-policies%252F%26cancel_url%3Dhttps%3A%2F%2Fm.facebook.com%2Fdialog%2Fclose_window%2F%3Fapp_id%3D966242223397117%26connect%3D0%23_%3D_%26display%3Dtouch%26locale%3Den_US printed
 Facebook Log into your Facebook account to share. Email or Phone: Password: Forgot account? Create New Account        
https://www.facebook.com/sharer/sharer.php?gfid=AQCzga_mJg46hI5U&index=2&locale2=en_US&lref=https://m.facebook.com/login.php?skip_api_login%3D1%26api_key%3D966242223397117%26signed_next%3D1%26next%3Dhttps%3A%2F%2Fm.facebook.com%2Fsharer%2Fsharer.php%3Fu%3Dhttps%253A%252F%252Fw

http://www.linkedin.com/shareArticle?fromSignIn=true&mini=true&trk=uno-reg-join-sign-in&url=https://www.oerlikon.com/fairfield/en/services/service-policies/https://www.linkedin.com/uas/login?session_redirect=https://www.linkedin.com/sharing/share-offsite?mini%3Dtrue%26url%3Dhttps://www.oerlikon.com/fairfield/en/services/service-policies/ printed
       LinkedIn Make the most of your professional life Not you?  Join LinkedIn First name Last name  Email Password (6 or more characters) You agree to the LinkedIn User Agreement , Privacy Policy , and Cookie Policy . Agree & Join or Agree & Continue with Facebook Already on LinkedIn? Sign in Join with Google Close Get set up quickly with Google. We'll receive: Your Email Address Your Google Profile: Name and Photo Your Gmail Contacts Use your Gmail contacts to find people you know on LinkedIn. You choose when and who to connect to. Information How secure is my information? We will never email any of your contacts until you specifically choos

          Sign in        To create a Google Account, you need JavaScript enabled in your browser.  Refresh this page after you have enabled JavaScript.     Create your Google Account                Name   First name   First    Last name   Last             Choose your username   @gmail.com   You can use letters, numbers, and periods.   I prefer to use my current email address      This email address already corresponds to a Google Account. Please sign in or, if you forgot your password, reset it .        Create a password       Password strength:        Use at least 8 characters. Don’t use a password from another site, or something too obvious like your pet’s name. Why?         Confirm your password         Birthday     Month  January  February  March  April  May  June  July  August  September  October  November  December      Day   Day    Year   Year             Gender    I am...  Female  Male  Other  Rather not say          Mobile phone          Your current email address    We will u

                 \uf1ad         Company Company Profile Company overview Organization Facts & Figures Market Segments Global footprint Compliance Tax strategy Sponsoring History Supplier Portal Contact Innovation Our Competencies Our Markets Research and Development Customer Reference Quality and Certificates Did you know 24h Trip 24h Around the World Oerlikon Blog Media Press Releases Events and Agenda Media Gallery Newsroom Subscribe RSS-Newsfeed Contact Newsletter Investor Relations Factsheet Corporate Governance Reports and Publications Share Information Creditor Relations Financial Calendar General Meeting Contact and Services Career Vacancies Why to join us How to join us Your Career with us Find your fit Our employees Contact Contact Headquarter Request form Worldwide        Group    Corporate    Surface Solutions Segment   Balzers  Metco  Additive Manufacturing     Manmade Fibers Segment   Barmag  Neumag     Drive Systems Segment   Graziano  Fairfield        Region   China  Ind

https://www.oerlikon.com/fairfield/en/ has been scraped successfully


In [ ]:
#executing the crawler function 

excel_path = 'batch1.xlsx'
sheet = 'Sheet1'
web_col = 2
index_col = 0
parent_col = 1
output_folder = 'Crawled\\'
from_website = 1
to_website = 2

web_crawler_all(excel_path,sheet, web_col, index_col, parent_col,output_folder, from_website, to_website)

   Serial No.             Company Name  \
0           6         Grand Industrial   
1           7  Harrison Steel Castings   

                                  Website  Has a Website?\n(1=Yes, 0=No)  \
0  https://www.oerlikon.com/fairfield/en/                              1   
1                  http://www.hscast.com/                              1   

   Static/Dynamic Website?  \
0                      NaN   
1                      NaN   

   If no website- Is present on Facebook?\n(1=Yes, 0=No)  Type of Company  \
0                                                NaN                  NaN   
1                                                NaN                  NaN   

   Products  Remarks  Certifications  static/dynamic? Vamsi!  Equipment Used  \
0       NaN      NaN             NaN                     NaN             NaN   
1       NaN      NaN             NaN                     NaN             NaN   

   Comments/ vamsi  
0              NaN  
1              NaN  
www.oerlikon.com


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://www.oerlikon.com/en/company/company-overview/surface-solutions/?dyk24=true/en/
https://www.oerlikon.com/en/company/company-overview/surface-solutions/?dyk24=true/en/portrait/
https://www.oerlikon.com/en/company/company-overview/surface-solutions/?dyk24=true/en/company/company-profile/
https://www.oerlikon.com/en/company/company-overview/surface-solutions/?dyk24=true/en/company/company-overview/
https://www.oerlikon.com/en/company/company-overview/surface-solutions/?dyk24=true/en/company/organization/board-of-directors/
https://www.oerlikon.com/en/company/company-overview/surface-solutions/?dyk24=true/en/company/key-figures/
https://www.oerlikon.com/en/company/company-overview/surface-solutions/?dyk24=true/en/company/market-segments/
https://www.oerlikon.com/en/company/company-overview/surface-solutions/?dyk24=true/en/company/worldwide-locations/
https://www.oerlikon.com/en/company/company-overview/surface-solutions/?dyk24=true/en/company/compliance/
https://www.oerlikon.com/en/

In [34]:
def scrape1(website_name):
    TempD={}
    #for index,item in df.iterrows():
    extensions=('.png', '.jpg','.jpeg','.pdf', '.gif','.ico')    #file_save(item[1],item[0])
    try:      
        print(website_name)
        print( website_name.endswith("extensions"))
        if 1:
              # print(item[1])
            html = urllib.request.urlopen(website_name).read()
            #print(html)
             #web_text.append(text_from_html(html))
            print(text_from_html(html))
            #    time.sleep(3)
                #save_url(str(item[1]), index)
           # TempD[str(item[1])]=text_from_html(html)
    except: 
        pass
   # with open(scraped_website_path + '/' + 'json_' + str(website_index) + str(get_base_url(website_name)) +  '.json', 'w') as outfile:  
       # json.dump(TempD, outfile)
        
    return text_from_html(html)